<a href="https://www.kaggle.com/code/soumikniloy/exercise-analytic-functions?scriptVersionId=140352212" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-yorcxvlh
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-yorcxvlh
  Resolved https://github.com/Kaggle/learntools.git to commit 93cf102cac95c2d8d7fb96ccc00ab5acc2c4bf05
  Preparing metadata (setup.py) ... - \ done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268981 sha256=92b4eccadb70fc11a52f643479afc40f367cba97fa235e03cd974b56c910a86f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1jfph282/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,01e6d039adfc5b0860ba59cc07481d5cec1aabb5,e8726856e228beb77e6dec4092366dd7a033327bf354c7...,2015-02-20 14:15:00+00:00,2015-02-20 15:00:00+00:00,2580,9.1,1.703124e+10,1.703115e+10,24,15,...,0.0,25.05,No Charge,Dispatch Taxi Affiliation,41.906651,-87.665338,POINT (-87.6653376596 41.9066507664),41.960676,-87.752409,POINT (-87.7524091367 41.9606756968)
1,3dafb65f7aae489dcf7f5e46472338b371002388,54a439b70c8a1f48a490f2c6018f552fbc9d99f25cbce2...,2015-02-16 12:00:00+00:00,2015-02-16 12:15:00+00:00,240,0.0,1.703106e+10,1.703183e+10,6,6,...,0.0,4.65,Cash,Blue Ribbon Taxi Association Inc.,41.936087,-87.666111,POINT (-87.6661106945 41.9360865352),41.945170,-87.668794,POINT (-87.6687944391 41.9451704528)
2,2294fa9344c817b8085e1500abe81b73cc8f936f,80c53adab72eacea6fbb1e58c0a95a2c665335149da7fe...,2015-02-20 14:30:00+00:00,2015-02-20 14:30:00+00:00,0,0.1,NaN,NaN,61,61,...,0.0,3.45,Cash,Taxi Affiliation Services,41.809018,-87.659167,POINT (-87.6591665992 41.8090182499),41.809018,-87.659167,POINT (-87.6591665992 41.8090182499)
3,dc20653b2ae30411cc9d6dee8b7a421be3d69c11,ba569986c43f453eecf59ffa5e73855a766c48a39cd9be...,2015-02-20 20:15:00+00:00,2015-02-20 20:15:00+00:00,240,0.8,1.703107e+10,1.703107e+10,7,7,...,1.0,8.25,Credit Card,KOAM Taxi Association,41.921855,-87.646211,POINT (-87.6462109769 41.9218549112),41.929263,-87.635891,POINT (-87.6358909539 41.9292629902)
4,a15b7c4f3d0fed18a7d877d9259a4a6904bbb94d,8f19a458e8774fb78d80233ea8a9e74f797282eb5869d9...,2015-03-13 23:15:00+00:00,2015-03-13 23:30:00+00:00,660,1.1,1.703106e+10,1.703106e+10,6,6,...,0.0,7.45,Cash,Taxi Affiliation Services,41.949829,-87.643965,POINT (-87.6439653703 41.949829346),41.934540,-87.643023,POINT (-87.6430228044 41.93453971580001)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-03-24,86435.714286
1,2016-03-18,95345.142857
2,2016-03-04,93811.000000
3,2016-01-29,79909.857143
4,2016-01-16,84642.285714


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,68.0,2013-10-03 05:15:00+00:00,2013-10-03 05:30:00+00:00,1
1,68.0,2013-10-03 10:30:00+00:00,2013-10-03 10:30:00+00:00,2
2,68.0,2013-10-03 10:30:00+00:00,2013-10-03 10:30:00+00:00,2
3,68.0,2013-10-03 14:45:00+00:00,2013-10-03 14:45:00+00:00,4
4,68.0,2013-10-03 15:15:00+00:00,2013-10-03 15:30:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

break_time_result = client.query(break_time_query).result().to_dataframe()

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,02e1118efc46c0d7c33d9bbf3fff0f8d8857584aaf73a4...,2013-10-03 18:30:00+00:00,2013-10-03 18:30:00+00:00,360.0
1,02e1118efc46c0d7c33d9bbf3fff0f8d8857584aaf73a4...,2013-10-03 22:30:00+00:00,2013-10-03 22:30:00+00:00,210.0
2,0737b02585acbf58cde0f9fe664d7cdaa2d67b29bf4f23...,2013-10-03 12:15:00+00:00,2013-10-03 12:15:00+00:00,-105.0
3,0737b02585acbf58cde0f9fe664d7cdaa2d67b29bf4f23...,2013-10-03 23:15:00+00:00,2013-10-03 23:30:00+00:00,390.0
4,0a89c89d2e743b2f137de7f7a068a45ec486cef6dba0d4...,2013-10-03 15:00:00+00:00,2013-10-03 15:15:00+00:00,165.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*